<a href="https://colab.research.google.com/github/yashas-anil/election-flippers/blob/main/2024_election_flippers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# You Can Flip Any Election by Moving People Around

In [ ]:
import pandas as pd

In [ ]:
# data source: 2024 election results, wikipedia
url = "https://en.wikipedia.org/wiki/2024_United_States_presidential_election"

In [ ]:
# for some reason there was an http error so I have to pretend to be a browser
import requests

header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}

r = requests.get(url, headers=header)

tables = pd.read_html(r.text) # array of all dataframes

/tmp/ipython-input-3480215772.py:11: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(r.text) # array of all dataframes


In [ ]:
df = tables[24]
df

State or district  Trump/Vance Republican  \
                                    State or district                   Votes   
0                                        Alabama[483]                 1462616   
1                                         Alaska[484]                  184458   
2                                        Arizona[485]                 1770242   
3                                       Arkansas[486]                  759241   
4                                     California[487]                 6081697   
5                                       Colorado[488]                 1377441   
6                                    Connecticut[489]                  736918   
7                                  Delaware[490][488]                  214351   
8                           District of Columbia[491]                   21076   
9                                        Florida[492]                 6110125   
10                                       Georgia[493]                 2663117   
11                                        Hawaii[494]                  193661   
12                                    Idaho[495][496]                  605246   
13                                 Illinois[497][488]                 2449079   
14                                       Indiana[488]                 1720347   
15                                          Iowa[498]                  927019   
16                                        Kansas[499]                  758802   
17                                      Kentucky[488]                 1337494   
18                                     Louisiana[500]                 1208505   
19                                  Maine †[501][488]                  377977   
20  ME-1Tooltip Maine's 1st congressional district...                  165214   
21  ME-2Tooltip Maine's 2nd congressional district...                  212763   
22                                      Maryland[502]                 1035550   
23                            Massachusetts[503][488]                 1251303   
24                                      Michigan[504]                 2816636   
25                                     Minnesota[505]                 1519032   
26                                   Mississippi[506]                  747744   
27                                      Missouri[507]                 1751986   
28                                       Montana[488]                  352079   
29                               Nebraska †[508][488]                  564816   
30  NE-1Tooltip Nebraska's 1st congressional distr...                  177666   
31  NE-2Tooltip Nebraska's 2nd congressional distr...                  148905   
32  NE-3Tooltip Nebraska's 3rd congressional distr...                  238245   
33                                        Nevada[509]                  751205   
34                                 New Hampshire[510]                  395523   
35                                    New Jersey[511]                 1968215   
36                                    New Mexico[512]                  423391   
37                                      New York[513]                 3578899   
38                                North Carolina[514]                 2898423   
39                                  North Dakota[515]                  246505   
40                                          Ohio[516]                 3180116   
41                                      Oklahoma[517]                 1036213   
42                                        Oregon[518]                  919480   
43                                  Pennsylvania[519]                 3543308   
44                                  Rhode Island[520]                  214406   
45                                South Carolina[521]                 1483747   
46                                  South Dakota[522]                  272081   
47                                     Tennessee[523]              

# Data Transformation
1. Cleaning the States column, dropping unneeded columns/rows, renaming columns for simplicity
2. Convert all numeric columns from string to int
3. Extract and drop national results
4. Add VPP column

In [ ]:
# remove the [] artifacts
import re

# Example column name
col = "State or district"
states = df[col][col]


# Remove square brackets and anything inside them
states = (
    states
    .str.replace(r"\[.*?\]", "", regex=True)          # remove [..]
    .str.replace(r"Tooltip.*", "", regex=True)        # remove Tooltip + following text
    .str.replace("†", "", regex=False)                # remove dagger symbol
    .str.strip()                                      # remove leading/trailing spaces
)


df.insert(1, "States", states)
df=df.drop("State or district", axis=1)

# remove unneeded columns, row, renamed for ease
df=df.drop(["Stein/Ware Green","Oliver/Maat Libertarian","Kennedy/Shanahan Independent","Others","Margin swing[b]"],axis=1)
df=df.drop(57)
df = df.rename(columns={"Trump/Vance Republican": 'R', 'Harris/Walz Democratic': 'D'})

/tmp/ipython-input-565833310.py:20: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df=df.drop("State or district", axis=1)
/tmp/ipython-input-565833310.py:23: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df=df.drop(["Stein/Ware Green","Oliver/Maat Libertarian","Kennedy/Shanahan Independent","Others","Margin swing[b]"],axis=1)


In [ ]:
# convert votes, %, EV from str to int
# apparently the minus signs are not the correct symbol, which is tripping up the str to int coversion, so we have to fix those dashes

num_df=df.copy(deep=True) # create new dataframe for simplicity

DASHES = "\u2212\u2012\u2013\u2014\u2015"  # minus, figure dash, en dash, em dash, horiz bar
num_cols=["R","D","Margin"]
for col in num_cols:
  s=df[col]['Votes']
  s = (
    s.str.strip()
    # Normalize any Unicode dash to ASCII hyphen-minus
    .str.replace(f"[{DASHES}]", "-", regex=True)
    # Remove commas used as thousands separators
    .str.replace(",", "", regex=False)
  )
  nums = pd.to_numeric(s, errors="coerce")

  # Convert to nullable integer type
  num_df.loc[:,(col,'Votes')] = nums.astype("Int64") # truncates toward 0 (e.g., -1.9 -> -1)

  # %
  p=df[col]['%']
  p = (
    p.str.strip()
    # Normalize any Unicode dash to ASCII hyphen-minus
    .str.replace(f"[{DASHES}]", "-", regex=True)
    # Remove %
    .str.replace("%", "", regex=False)
  )
  nums = pd.to_numeric(p, errors="coerce")

  # Convert to nullable integer type
  num_df.loc[:,(col,'%')] = nums.astype("Float64") # truncates toward 0 (e.g., -1.9 -> -1)


  #  EV
  if col!='Margin': # margin doesnt have an EV col
    e=df[col]['EV']
    e=e.str.replace(f"[{DASHES}]", "0", regex=True) # the "nulls" are just dashes so we can replace them with 0s
    nums = pd.to_numeric(e, errors="coerce")
    num_df.loc[:,(col,'EV')] = nums.astype("Int64")

#convert Total
num_df.rename(columns={"Total votes":"Total"},inplace=True)
col="Total votes"
s=df[col][col]
nums = pd.to_numeric(s, errors="coerce")
num_df['Total Votes'] = nums.astype("Int64")
num_df=num_df.drop("Total", axis=1) #drop the old total

num_df

/tmp/ipython-input-353240697.py:50: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  num_df=num_df.drop("Total", axis=1) #drop the old total


States         R                     D              \
                             Votes      %   EV     Votes      %   EV   
0                Alabama   1462616  64.57    9    772412   34.1    0   
1                 Alaska    184458  54.54    3    140026  41.41    0   
2                Arizona   1770242  52.22   11   1582860  46.69    0   
3               Arkansas    759241   64.2    6    396905  33.56    0   
4             California   6081697  38.33    0   9276179  58.47   54   
5               Colorado   1377441  43.14    0   1728159  54.13   10   
6            Connecticut    736918  41.89    0    992053   56.4    7   
7               Delaware    214351  41.79    0    289758  56.49    3   
8   District of Columbia     21076   6.47    0    294185  90.28    3   
9                Florida   6110125  56.09   30   4683038  42.99    0   
10               Georgia   2663117  50.72   16   2548017  48.53    0   
11                Hawaii    193661  37.48    0    313044  60.59    4   
12                 Idaho    605246  66.87    4    274972  30.38    0   
13              Illinois   2449079  43.47    0   3062863  54.37   19   
14               Indiana   1720347  58.58   11   1163603  39.62    0   
15                  Iowa    927019  55.73    6    707278  42.52    0   
16                Kansas    758802  57.16    6    544853  41.04    0   
17              Kentucky   1337494  64.47    8    704043  33.94    0   
18             Louisiana   1208505  60.22    8    766870  38.21    0   
19                 Maine    377977  45.46    0    435652   52.4    2   
20                  ME-1    165214  38.09    0    258863  59.69    1   
21                  ME-2    212763   53.5    1    176789  44.46    0   
22              Maryland   1035550  34.08    0   1902577  62.62   10   
23         Massachusetts   1251303  36.02    0   2126518  61.22   11   
24              Michigan   2816636  49.73   15   2736533  48.31    0   
25             Minnesota   1519032  46.68    0   1656979  50.92   10   
26           Mississippi    747744  60.89    6    466668   38.0    0   
27              Missouri   1751986  58.49   10   1200599  40.08    0   
28               Montana    352079  58.39    4    231906  38.46    0   
29              Nebraska    564816  59.32    2    369995  38.86    0   
30                  NE-1    177666  55.49    1    136153  42.52    0   
31                  NE-2    148905  46.73    0    163541  51.32    1   
32                  NE-3    238245  76.03    1     70301  22.44    0   
33                Nevada    751205  50.59    6    705197  47.49    0   
34         New Hampshire    395523  47.87    0    418488  50.65    4   
35            New Jersey   1968215  46.06    0   2220713  51.97   14   
36            New Mexico    423391  45.85    0    478802  51.85    5   
37              New York   3578899  43.31    0   4619195  55.91   28   
38        North Carolina   2898423  50.86   16   2715375  47.65    0   
39          North Dakota    246505  66.96    3    112327  30.51    0   
40                  Ohio   3180116  55.14   17   2533699  43.93    0   
41              Oklahoma   1036213  66.16    7    499599   31.9    0   
42                Oregon    919480  40.97    0   1240600  55.27    8   
43          Pennsylvania   3543308  50.37   19   3423042  48.66    0   
44          Rhode Island    214406  41.76    0    285156  55.54    4   
45        South Carolina   1483747  58.23    9   1028452  40.36    0   
46          South Dakota    272081  63.43    3    146859  34.24    0   
47             Tennessee   1966865  64.19   11   1056265  34.47    0   
48                 Texas   6393597  56.14   40   4835250  42.46    0   
49                  Utah    883818  59.38    6    562566  37.79    0   
50               Vermont    119395  32.32    0    235791  63.83    3   
51              Virginia   2075085  46.05    0   2335395  51.83   13   
52            Washington   1530923  39.01    0   2245849  57.23   12   
53         West Virginia    533556  69.97    4    214309  

In [ ]:
# the total row is the popular vote, so we'll extract that and keep it separate
national=num_df.iloc[56]
num_df=num_df.drop(56)
national

States                    Total
R            Votes     77302580
             %             49.8
             EV             312
D            Votes     75017613
             %            48.32
             EV             226
Margin       Votes      2284967
             %             1.48
Total Votes           155238302
Name: 56, dtype: object

Voters per Point (VPP) is a metric to determine whether a state should be flipped. It is defined as Margin(voters)/StateEV, and tells us how many voters each flipped EV point "costs". Therefore we want to start our flipping algorithm on those with the lowest VPP, which have the most value for their cost. Additionally, when using an underflow algorithm, we want to sacrifice our high VPP states, because they are the least worth flipping.

In [ ]:
num_df['VPP']=num_df["Margin"]['Votes']/(num_df["D"]['EV']+num_df["R"]['EV'])
num_df['VPP']=num_df['VPP'].astype(float).round(2)
# sort_df=num_df.sort_values(by='VPP')

# Testing Section
This is just for experiments, and will be deleted upon completion

In [ ]:
test_df=num_df.copy(deep=True)
test_df.loc[:,("D","%")]=(test_df.loc[:,("D","Votes")]/test_df.loc[:,"Total Votes"]).astype(float).round(2)

In [ ]:
test_df.loc[test_df.index[9],("Margin","Votes")]=101
test_df[("Margin","Votes")].iloc[8]
test_df.loc[1,"Total Votes"]=int(test_df.loc[1,"Total Votes"])*2
test_df

In [ ]:
test_df.loc[1,"Total Votes"]=100
int(test_df["Total Votes"].iloc[1])

#Flipping Algorithm
1.   Calculate Minimum EV needed to flip
2.   Overflow algorithm (Minimum and Primary)
3.   Underflow algorithm (Minimum)
4.   For each algo, begin with copies of the winners and losers dataframes, and after the algo has run, concatenate them back together


Algorithm also has to keep track of votes being siphoned from source states

In [ ]:
# Minimum EV
import math
nat_total_EV=national[("R","EV")]+national[("D","EV")]
if nat_total_EV%2==0:
  threshold_EV=int(nat_total_EV/2)+1
else:
  threshold_EV=math.ceil(nat_total_EV/2)
minimum_EV=threshold_EV-national[("D","EV")] #the minimum EV to flip to flip the entire election
minimum_EV

44

In [ ]:
losers_df=num_df[num_df['Margin']['Votes']<0]
losers_df=losers_df.sort_values(by=[('Margin',"Votes")])
losers_df

States        R                  D               Margin  \
                            Votes      % EV    Votes      %  EV     Votes   
4             California  6081697  38.33  0  9276179  58.47  54  -3194482   
37              New York  3578899  43.31  0  4619195  55.91  28  -1040296   
23         Massachusetts  1251303  36.02  0  2126518  61.22  11   -875215   
22              Maryland  1035550  34.08  0  1902577  62.62  10   -867027   
52            Washington  1530923  39.01  0  2245849  57.23  12   -714926   
13              Illinois  2449079  43.47  0  3062863  54.37  19   -613784   
5               Colorado  1377441  43.14  0  1728159  54.13  10   -350718   
42                Oregon   919480  40.97  0  1240600  55.27   8   -321120   
8   District of Columbia    21076   6.47  0   294185  90.28   3   -273109   
51              Virginia  2075085  46.05  0  2335395  51.83  13   -260310   
6            Connecticut   736918  41.89  0   992053   56.4   7   -255135   
35            New Jersey  1968215  46.06  0  2220713  51.97  14   -252498   
25             Minnesota  1519032  46.68  0  1656979  50.92  10   -137947   
11                Hawaii   193661  37.48  0   313044  60.59   4   -119383   
50               Vermont   119395  32.32  0   235791  63.83   3   -116396   
20                  ME-1   165214  38.09  0   258863  59.69   1    -93649   
7               Delaware   214351  41.79  0   289758  56.49   3    -75407   
44          Rhode Island   214406  41.76  0   285156  55.54   4    -70750   
19                 Maine   377977  45.46  0   435652   52.4   2    -57675   
36            New Mexico   423391  45.85  0   478802  51.85   5    -55411   
34         New Hampshire   395523  47.87  0   418488  50.65   4    -22965   
31                  NE-2   148905  46.73  0   163541  51.32   1    -14636   

          Total Votes       VPP  
        %                        
4  -20.14    15865475 -59157.07  
37  -12.6     8262495 -37153.43  
23  -25.2     3473668 -79565.00  
22 -28.54     3038334 -86702.70  
52 -18.22     3924243 -59577.17  
13  -10.9     5633310 -32304.42  
5  -10.99     3192745 -35071.80  
42  -14.3     2244493 -40140.00  
8  -83.81      325869 -91036.33  
51  -5.78     4505941 -20023.85  
6  -14.51     1759010 -36447.86  
35  -5.91     4272725 -18035.57  
25  -4.24     3253920 -13794.70  
11 -23.11      516701 -29845.75  
50 -31.51      369422 -38798.67  
20  -21.6      433709 -93649.00  
7   -14.7      512912 -25135.67  
44 -13.78      513386 -17687.50  
19  -6.94      831375 -28837.50  
36   -6.0      923403 -11082.20  
34  -2.78      826189  -5741.25  
31  -4.59      318646 -14636.00

In [ ]:
winners_df=num_df[num_df['Margin']['Votes']>0]
winners_df=winners_df.sort_values(by='VPP')
winners_df.reset_index(inplace=True) #reset index to put low VPP at top
winners_df.drop("index", axis=1, inplace=True) #drop original mixed up index
winners_df

/tmp/ipython-input-1642076033.py:4: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  winners_df.drop("index", axis=1, inplace=True) #drop original mixed up index


States        R                   D             Margin         \
                      Votes      %  EV    Votes      % EV    Votes      %   
0        Wisconsin  1697626   49.6  10  1668229  48.74  0    29397   0.86   
1         Michigan  2816636  49.73  15  2736533  48.31  0    80103   1.42   
2     Pennsylvania  3543308  50.37  19  3423042  48.66  0   120266   1.71   
3          Georgia  2663117  50.72  16  2548017  48.53  0   115100   2.19   
4           Nevada   751205  50.59   6   705197  47.49  0    46008    3.1   
5   North Carolina  2898423  50.86  16  2715375  47.65  0   183048   3.21   
6           Alaska   184458  54.54   3   140026  41.41  0    44432  13.13   
7          Arizona  1770242  52.22  11  1582860  46.69  0   187382   5.53   
8          Montana   352079  58.39   4   231906  38.46  0   120173  19.93   
9           Kansas   758802  57.16   6   544853  41.04  0   213949  16.12   
10            ME-2   212763   53.5   1   176789  44.46  0    35974   9.05   
11            Iowa   927019  55.73   6   707278  42.52  0   219741  13.21   
12            Ohio  3180116  55.14  17  2533699  43.93  0   646417  11.21   
13           Texas  6393597  56.14  40  4835250  42.46  0  1558347  13.68   
14         Wyoming   192633   71.6   3    69527  25.84  0   123106  45.76   
15            NE-1   177666  55.49   1   136153  42.52  0    41513  12.96   
16    South Dakota   272081  63.43   3   146859  34.24  0   125222  29.19   
17    North Dakota   246505  66.96   3   112327  30.51  0   134178  36.45   
18     Mississippi   747744  60.89   6   466668   38.0  0   281076  22.89   
19         Florida  6110125  56.09  30  4683038  42.99  0  1427087   13.1   
20  South Carolina  1483747  58.23   9  1028452  40.36  0   455295  17.87   
21         Indiana  1720347  58.58  11  1163603  39.62  0   556744  18.96   
22            Utah   883818  59.38   6   562566  37.79  0   321252  21.59   
23        Missouri  1751986  58.49  10  1200599  40.08  0   551387  18.41   
24       Louisiana  1208505  60.22   8   766870  38.21  0   441635  22.01   
25        Arkansas   759241   64.2   6   396905  33.56  0   362336  30.64   
26        Oklahoma  1036213  66.16   7   499599   31.9  0   536614  34.26   
27         Alabama  1462616  64.57   9   772412   34.1  0   690204  30.47   
28        Kentucky  1337494  64.47   8   704043  33.94  0   633451  30.53   
29   West Virginia   533556  69.97   4   214309   28.1  0   319247  41.87   
30           Idaho   605246  66.87   4   274972  30.38  0   330274  36.49   
31       Tennessee  1966865  64.19  11  1056265  34.47  0   910600  29.72   
32        Nebraska   564816  59.32   2   369995  38.86  0   194821  20.46   
33            NE-3   238245  76.03   1    70301  22.44  0   167944   53.6   

   Total Votes        VPP  
                           
0      3422918    2939.70  
1      5664186    5340.20  
2      7058732    6329.79  
3      5250905    7193.75  
4      1484840    7668.00  
5      5699141   11440.50  
6       338177   14810.67  
7      3390161   17034.73  
8       602990   30043.25  
9      1327591   35658.17  
10      397666   35974.00  
11     1663506   36623.50  
12     5767788   38024.53  
13    11388674   38958.68  
14      269048   41035.33  
15      320194   41513.00  
16      428922   41740.67  
17      368155   44726.00  
18     1228008   46846.00  
19    10893752   47569.57  
20     2548140   50588.33  
21     2936677   50613.09  
22     1488494   53542.00  
23     2995327   55138.70  
24     2006975   55204.38  
25     1182676   60389.33  
26     1566173   76659.14  
27     2265090   76689.33  
28     2074530   79181.38  
29      762582   79811.75  
30      905057   82568.50  
31     3063942   82781.82  
32      952182   97410.50  
33      313342  167944.00

# Overflow Algorithm: Minimum EV
The primary source state is the state from the og loser with the most votes to spare (overflow). In 2024 it was California.

Because the outcome is flipped, the copy of losers will be called "flip winners" and the copy of winners will be "flip losers"

Specifically, as this first algorithm will just flip the minimum EV and will use overflow method, the copies will be called "min_over_winners" and "min_over_losers"

In [ ]:
min_over_winners_df=losers_df.copy(deep=True)

min_over_losers_df=winners_df.copy(deep=True)

min_over_losers_df #so we can compare it to the outcome of the algo

States        R                   D             Margin         \
                      Votes      %  EV    Votes      % EV    Votes      %   
0        Wisconsin  1697626   49.6  10  1668229  48.74  0    29397   0.86   
1         Michigan  2816636  49.73  15  2736533  48.31  0    80103   1.42   
2     Pennsylvania  3543308  50.37  19  3423042  48.66  0   120266   1.71   
3          Georgia  2663117  50.72  16  2548017  48.53  0   115100   2.19   
4           Nevada   751205  50.59   6   705197  47.49  0    46008    3.1   
5   North Carolina  2898423  50.86  16  2715375  47.65  0   183048   3.21   
6           Alaska   184458  54.54   3   140026  41.41  0    44432  13.13   
7          Arizona  1770242  52.22  11  1582860  46.69  0   187382   5.53   
8          Montana   352079  58.39   4   231906  38.46  0   120173  19.93   
9           Kansas   758802  57.16   6   544853  41.04  0   213949  16.12   
10            ME-2   212763   53.5   1   176789  44.46  0    35974   9.05   
11            Iowa   927019  55.73   6   707278  42.52  0   219741  13.21   
12            Ohio  3180116  55.14  17  2533699  43.93  0   646417  11.21   
13           Texas  6393597  56.14  40  4835250  42.46  0  1558347  13.68   
14         Wyoming   192633   71.6   3    69527  25.84  0   123106  45.76   
15            NE-1   177666  55.49   1   136153  42.52  0    41513  12.96   
16    South Dakota   272081  63.43   3   146859  34.24  0   125222  29.19   
17    North Dakota   246505  66.96   3   112327  30.51  0   134178  36.45   
18     Mississippi   747744  60.89   6   466668   38.0  0   281076  22.89   
19         Florida  6110125  56.09  30  4683038  42.99  0  1427087   13.1   
20  South Carolina  1483747  58.23   9  1028452  40.36  0   455295  17.87   
21         Indiana  1720347  58.58  11  1163603  39.62  0   556744  18.96   
22            Utah   883818  59.38   6   562566  37.79  0   321252  21.59   
23        Missouri  1751986  58.49  10  1200599  40.08  0   551387  18.41   
24       Louisiana  1208505  60.22   8   766870  38.21  0   441635  22.01   
25        Arkansas   759241   64.2   6   396905  33.56  0   362336  30.64   
26        Oklahoma  1036213  66.16   7   499599   31.9  0   536614  34.26   
27         Alabama  1462616  64.57   9   772412   34.1  0   690204  30.47   
28        Kentucky  1337494  64.47   8   704043  33.94  0   633451  30.53   
29   West Virginia   533556  69.97   4   214309   28.1  0   319247  41.87   
30           Idaho   605246  66.87   4   274972  30.38  0   330274  36.49   
31       Tennessee  1966865  64.19  11  1056265  34.47  0   910600  29.72   
32        Nebraska   564816  59.32   2   369995  38.86  0   194821  20.46   
33            NE-3   238245  76.03   1    70301  22.44  0   167944   53.6   

   Total Votes        VPP  
                           
0      3422918    2939.70  
1      5664186    5340.20  
2      7058732    6329.79  
3      5250905    7193.75  
4      1484840    7668.00  
5      5699141   11440.50  
6       338177   14810.67  
7      3390161   17034.73  
8       602990   30043.25  
9      1327591   35658.17  
10      397666   35974.00  
11     1663506   36623.50  
12     5767788   38024.53  
13    11388674   38958.68  
14      269048   41035.33  
15      320194   41513.00  
16      428922   41740.67  
17      368155   44726.00  
18     1228008   46846.00  
19    10893752   47569.57  
20     2548140   50588.33  
21     2936677   50613.09  
22     1488494   53542.00  
23     2995327   55138.70  
24     2006975   55204.38  
25     1182676   60389.33  
26     1566173   76659.14  
27     2265090   76689.33  
28     2074530   79181.38  
29      762582   79811.75  
30      905057   82568.50  
31     3063942   82781.82  
32      952182   97410.50  
33      313342  167944.00

In [ ]:
n=0
flipped_EV=0
total_voters_moved=0 #the number of voters moved overall
state_voters_moved=0 #the number of voters moved into each state to flip it
mini_margin=1000  #our arbitrary minimum acceptable margin of statewide victory
source_state=min_over_winners_df.iloc[0]
overflow=source_state.loc[("Margin","Votes")]*(-1)-mini_margin

while flipped_EV<minimum_EV:
  state_voters_moved=min_over_losers_df.loc[n,("Margin","Votes")]+mini_margin
  total_voters_moved=total_voters_moved+state_voters_moved
  overflow=overflow-state_voters_moved

  # change votes
  min_over_losers_df.loc[n,("D","Votes")]=int(min_over_losers_df.loc[n,("D","Votes")])+state_voters_moved
  min_over_losers_df.loc[n,"Total Votes"]=int(min_over_losers_df["Total Votes"].iloc[n])+state_voters_moved
  min_over_losers_df.loc[n,("Margin","Votes")]=mini_margin*(-1)
  # NOTE: for some godforsaken reason we have to use int(flip_losers_df["Total Votes"].iloc[n]) to GET the total votes but flip_losers_df.loc[flip_losers_df.index[n],"Total Votes"] when setting the total votes!
  # generally it seems .loc is needed for set but just [] is fine for accessing, but there's a bunch of caveats ofc

  # change EV
  flipped_EV=flipped_EV+min_over_losers_df.loc[n,("R","EV")]
  min_over_losers_df.loc[n,("D","EV")]=min_over_losers_df.loc[n,("R","EV")]
  min_over_losers_df.loc[n,("R","EV")]=0

  n=n+1

min_over_losers_df


States        R                   D              Margin         \
                      Votes      %  EV    Votes      %  EV    Votes      %   
0        Wisconsin  1697626   49.6   0  1698626  48.74  10    -1000   0.86   
1         Michigan  2816636  49.73   0  2817636  48.31  15    -1000   1.42   
2     Pennsylvania  3543308  50.37   0  3544308  48.66  19    -1000   1.71   
3          Georgia  2663117  50.72  16  2548017  48.53   0   115100   2.19   
4           Nevada   751205  50.59   6   705197  47.49   0    46008    3.1   
5   North Carolina  2898423  50.86  16  2715375  47.65   0   183048   3.21   
6           Alaska   184458  54.54   3   140026  41.41   0    44432  13.13   
7          Arizona  1770242  52.22  11  1582860  46.69   0   187382   5.53   
8          Montana   352079  58.39   4   231906  38.46   0   120173  19.93   
9           Kansas   758802  57.16   6   544853  41.04   0   213949  16.12   
10            ME-2   212763   53.5   1   176789  44.46   0    35974   9.05   
11            Iowa   927019  55.73   6   707278  42.52   0   219741  13.21   
12            Ohio  3180116  55.14  17  2533699  43.93   0   646417  11.21   
13           Texas  6393597  56.14  40  4835250  42.46   0  1558347  13.68   
14         Wyoming   192633   71.6   3    69527  25.84   0   123106  45.76   
15            NE-1   177666  55.49   1   136153  42.52   0    41513  12.96   
16    South Dakota   272081  63.43   3   146859  34.24   0   125222  29.19   
17    North Dakota   246505  66.96   3   112327  30.51   0   134178  36.45   
18     Mississippi   747744  60.89   6   466668   38.0   0   281076  22.89   
19         Florida  6110125  56.09  30  4683038  42.99   0  1427087   13.1   
20  South Carolina  1483747  58.23   9  1028452  40.36   0   455295  17.87   
21         Indiana  1720347  58.58  11  1163603  39.62   0   556744  18.96   
22            Utah   883818  59.38   6   562566  37.79   0   321252  21.59   
23        Missouri  1751986  58.49  10  1200599  40.08   0   551387  18.41   
24       Louisiana  1208505  60.22   8   766870  38.21   0   441635  22.01   
25        Arkansas   759241   64.2   6   396905  33.56   0   362336  30.64   
26        Oklahoma  1036213  66.16   7   499599   31.9   0   536614  34.26   
27         Alabama  1462616  64.57   9   772412   34.1   0   690204  30.47   
28        Kentucky  1337494  64.47   8   704043  33.94   0   633451  30.53   
29   West Virginia   533556  69.97   4   214309   28.1   0   319247  41.87   
30           Idaho   605246  66.87   4   274972  30.38   0   330274  36.49   
31       Tennessee  1966865  64.19  11  1056265  34.47   0   910600  29.72   
32        Nebraska   564816  59.32   2   369995  38.86   0   194821  20.46   
33            NE-3   238245  76.03   1    70301  22.44   0   167944   53.6   

   Total Votes        VPP  
                           
0      3453315    2939.70  
1      5745289    5340.20  
2      7179998    6329.79  
3      5250905    7193.75  
4      1484840    7668.00  
5      5699141   11440.50  
6       338177   14810.67  
7      3390161   17034.73  
8       602990   30043.25  
9      1327591   35658.17  
10      397666   35974.00  
11     1663506   36623.50  
12     5767788   38024.53  
13    11388674   38958.68  
14      269048   41035.33  
15      320194   41513.00  
16      428922   41740.67  
17      368155   44726.00  
18     1228008   46846.00  
19    10893752   47569.57  
20     2548140   50588.33  
21     2936677   50613.09  
22     1488494   53542.00  
23     2995327   55138.70  
24     2006975   55204.38  
25     1182676   60389.33  
26     1566173   76659.14  
27     2265090   76689.33  
28     2074530   79181.38  
29      762582   79811.75  
30      905057   82568.50  
31     3063942   82781.82  
32      952182   97410.50  
33      313342  167944.00

In [ ]:
total_voters_moved

232766

# Overflow Algorithm: One Source State
Passing the minimum EV and taking as much from the primary source state (California) as possible

In [ ]:
over_winners_df=losers_df.copy(deep=True)

over_losers_df=winners_df.copy(deep=True)

over_losers_df #so we can compare it to the outcome of the algo

States        R                   D             Margin         \
                      Votes      %  EV    Votes      % EV    Votes      %   
0        Wisconsin  1697626   49.6  10  1668229  48.74  0    29397   0.86   
1         Michigan  2816636  49.73  15  2736533  48.31  0    80103   1.42   
2     Pennsylvania  3543308  50.37  19  3423042  48.66  0   120266   1.71   
3          Georgia  2663117  50.72  16  2548017  48.53  0   115100   2.19   
4           Nevada   751205  50.59   6   705197  47.49  0    46008    3.1   
5   North Carolina  2898423  50.86  16  2715375  47.65  0   183048   3.21   
6           Alaska   184458  54.54   3   140026  41.41  0    44432  13.13   
7          Arizona  1770242  52.22  11  1582860  46.69  0   187382   5.53   
8          Montana   352079  58.39   4   231906  38.46  0   120173  19.93   
9           Kansas   758802  57.16   6   544853  41.04  0   213949  16.12   
10            ME-2   212763   53.5   1   176789  44.46  0    35974   9.05   
11            Iowa   927019  55.73   6   707278  42.52  0   219741  13.21   
12            Ohio  3180116  55.14  17  2533699  43.93  0   646417  11.21   
13           Texas  6393597  56.14  40  4835250  42.46  0  1558347  13.68   
14         Wyoming   192633   71.6   3    69527  25.84  0   123106  45.76   
15            NE-1   177666  55.49   1   136153  42.52  0    41513  12.96   
16    South Dakota   272081  63.43   3   146859  34.24  0   125222  29.19   
17    North Dakota   246505  66.96   3   112327  30.51  0   134178  36.45   
18     Mississippi   747744  60.89   6   466668   38.0  0   281076  22.89   
19         Florida  6110125  56.09  30  4683038  42.99  0  1427087   13.1   
20  South Carolina  1483747  58.23   9  1028452  40.36  0   455295  17.87   
21         Indiana  1720347  58.58  11  1163603  39.62  0   556744  18.96   
22            Utah   883818  59.38   6   562566  37.79  0   321252  21.59   
23        Missouri  1751986  58.49  10  1200599  40.08  0   551387  18.41   
24       Louisiana  1208505  60.22   8   766870  38.21  0   441635  22.01   
25        Arkansas   759241   64.2   6   396905  33.56  0   362336  30.64   
26        Oklahoma  1036213  66.16   7   499599   31.9  0   536614  34.26   
27         Alabama  1462616  64.57   9   772412   34.1  0   690204  30.47   
28        Kentucky  1337494  64.47   8   704043  33.94  0   633451  30.53   
29   West Virginia   533556  69.97   4   214309   28.1  0   319247  41.87   
30           Idaho   605246  66.87   4   274972  30.38  0   330274  36.49   
31       Tennessee  1966865  64.19  11  1056265  34.47  0   910600  29.72   
32        Nebraska   564816  59.32   2   369995  38.86  0   194821  20.46   
33            NE-3   238245  76.03   1    70301  22.44  0   167944   53.6   

   Total Votes        VPP  
                           
0      3422918    2939.70  
1      5664186    5340.20  
2      7058732    6329.79  
3      5250905    7193.75  
4      1484840    7668.00  
5      5699141   11440.50  
6       338177   14810.67  
7      3390161   17034.73  
8       602990   30043.25  
9      1327591   35658.17  
10      397666   35974.00  
11     1663506   36623.50  
12     5767788   38024.53  
13    11388674   38958.68  
14      269048   41035.33  
15      320194   41513.00  
16      428922   41740.67  
17      368155   44726.00  
18     1228008   46846.00  
19    10893752   47569.57  
20     2548140   50588.33  
21     2936677   50613.09  
22     1488494   53542.00  
23     2995327   55138.70  
24     2006975   55204.38  
25     1182676   60389.33  
26     1566173   76659.14  
27     2265090   76689.33  
28     2074530   79181.38  
29      762582   79811.75  
30      905057   82568.50  
31     3063942   82781.82  
32      952182   97410.50  
33      313342  167944.00

In [ ]:
n=0
flipped_EV=0
total_voters_moved=0 #the number of voters moved overall
state_voters_moved=0 #the number of voters moved into each state to flip it
mini_margin=10000  #our arbitrary minimum acceptable margin of statewide victory
source_state=over_winners_df.iloc[0]
overflow=source_state.loc[("Margin","Votes")]*(-1)-mini_margin

while overflow>(over_losers_df.loc[n,("Margin","Votes")]+mini_margin):
  state_voters_moved=over_losers_df.loc[n,("Margin","Votes")]+mini_margin
  total_voters_moved=total_voters_moved+state_voters_moved
  overflow=overflow-state_voters_moved

  # change votes
  over_losers_df.loc[n,("D","Votes")]=int(over_losers_df.loc[n,("D","Votes")])+state_voters_moved
  over_losers_df.loc[n,"Total Votes"]=int(over_losers_df["Total Votes"].iloc[n])+state_voters_moved
  over_losers_df.loc[n,("Margin","Votes")]=mini_margin*(-1)
  # NOTE: for some godforsaken reason we have to use int(flip_losers_df["Total Votes"].iloc[n]) to GET the total votes but flip_losers_df.loc[flip_losers_df.index[n],"Total Votes"] when setting the total votes!
  # generally it seems .loc is needed for set but just [] is fine for accessing, but there's a bunch of caveats ofc

  # change EV
  flipped_EV=flipped_EV+over_losers_df.loc[n,("R","EV")]
  over_losers_df.loc[n,("D","EV")]=over_losers_df.loc[n,("R","EV")]
  over_losers_df.loc[n,("R","EV")]=0

  n=n+1

over_losers_df


States        R                   D              Margin         \
                      Votes      %  EV    Votes      %  EV    Votes      %   
0        Wisconsin  1697626   49.6   0  1707626  48.74  10   -10000   0.86   
1         Michigan  2816636  49.73   0  2826636  48.31  15   -10000   1.42   
2     Pennsylvania  3543308  50.37   0  3553308  48.66  19   -10000   1.71   
3          Georgia  2663117  50.72   0  2673117  48.53  16   -10000   2.19   
4           Nevada   751205  50.59   0   761205  47.49   6   -10000    3.1   
5   North Carolina  2898423  50.86   0  2908423  47.65  16   -10000   3.21   
6           Alaska   184458  54.54   0   194458  41.41   3   -10000  13.13   
7          Arizona  1770242  52.22   0  1780242  46.69  11   -10000   5.53   
8          Montana   352079  58.39   0   362079  38.46   4   -10000  19.93   
9           Kansas   758802  57.16   0   768802  41.04   6   -10000  16.12   
10            ME-2   212763   53.5   0   222763  44.46   1   -10000   9.05   
11            Iowa   927019  55.73   0   937019  42.52   6   -10000  13.21   
12            Ohio  3180116  55.14   0  3190116  43.93  17   -10000  11.21   
13           Texas  6393597  56.14  40  4835250  42.46   0  1558347  13.68   
14         Wyoming   192633   71.6   3    69527  25.84   0   123106  45.76   
15            NE-1   177666  55.49   1   136153  42.52   0    41513  12.96   
16    South Dakota   272081  63.43   3   146859  34.24   0   125222  29.19   
17    North Dakota   246505  66.96   3   112327  30.51   0   134178  36.45   
18     Mississippi   747744  60.89   6   466668   38.0   0   281076  22.89   
19         Florida  6110125  56.09  30  4683038  42.99   0  1427087   13.1   
20  South Carolina  1483747  58.23   9  1028452  40.36   0   455295  17.87   
21         Indiana  1720347  58.58  11  1163603  39.62   0   556744  18.96   
22            Utah   883818  59.38   6   562566  37.79   0   321252  21.59   
23        Missouri  1751986  58.49  10  1200599  40.08   0   551387  18.41   
24       Louisiana  1208505  60.22   8   766870  38.21   0   441635  22.01   
25        Arkansas   759241   64.2   6   396905  33.56   0   362336  30.64   
26        Oklahoma  1036213  66.16   7   499599   31.9   0   536614  34.26   
27         Alabama  1462616  64.57   9   772412   34.1   0   690204  30.47   
28        Kentucky  1337494  64.47   8   704043  33.94   0   633451  30.53   
29   West Virginia   533556  69.97   4   214309   28.1   0   319247  41.87   
30           Idaho   605246  66.87   4   274972  30.38   0   330274  36.49   
31       Tennessee  1966865  64.19  11  1056265  34.47   0   910600  29.72   
32        Nebraska   564816  59.32   2   369995  38.86   0   194821  20.46   
33            NE-3   238245  76.03   1    70301  22.44   0   167944   53.6   

   Total Votes        VPP  
                           
0      3462315    2939.70  
1      5754289    5340.20  
2      7188998    6329.79  
3      5376005    7193.75  
4      1540848    7668.00  
5      5892189   11440.50  
6       392609   14810.67  
7      3587543   17034.73  
8       733163   30043.25  
9      1551540   35658.17  
10      443640   35974.00  
11     1893247   36623.50  
12     6424205   38024.53  
13    11388674   38958.68  
14      269048   41035.33  
15      320194   41513.00  
16      428922   41740.67  
17      368155   44726.00  
18     1228008   46846.00  
19    10893752   47569.57  
20     2548140   50588.33  
21     2936677   50613.09  
22     1488494   53542.00  
23     2995327   55138.70  
24     2006975   55204.38  
25     1182676   60389.33  
26     1566173   76659.14  
27     2265090   76689.33  
28     2074530   79181.38  
29      762582   79811.75  
30      905057   82568.50  
31     3063942   82781.82  
32      952182   97410.50  
33      313342  167944.00

In [ ]:
total_voters_moved

2171990

In [ ]:
overflow

1012492

# Remaining objectives/to do

Minimum flip algorithm: Set minimum EV needed to flip and stop flip when minimum is hit

Overflow algo:
* **minimum EV**
* just the primary source
* * note: sometimes the minimum EV will require more than the primary source
* all source states

Underflow algo: start from the least flippable states (highest VPP)
*   **minimum EV**
*   List item




Merge flip losers and flip winners

If winning by one vote is considered too narrow, the algorithm could be changed to require a larger margin

# GENERALIZATION TO DO
Many features of the code need to be modified so that it can be run on any election, not just 2024
* Since different parties win different elections, references to column names need to be relaced with column indices.
  * The original column names can be put into a series that can be referenced later to specify the part/candidate names
  * After "states" the winner is typically the first column, then the loser, then the third parties, then margin, then margin swing, then total
